# Alberto Dicembre, Assignment #4

In [1]:
from bayesian_network import BayesianNetwork 


## Testing the Network

In [2]:
net = BayesianNetwork(values=["True", "False"]) 
net.add_node("A")
net.add_node("B")
net.add_node("C")


In [3]:
print(net)
net.check_valid()

Nodes: dict_keys(['A', 'B', 'C']), Edges: {}, Tables: {'A': {'Prior': (0.5, 0.5)}, 'B': {'Prior': (0.5, 0.5)}, 'C': {'Prior': (0.5, 0.5)}}


True

In [4]:
net.add_edge("A", ["B"])
net.add_edge("B", ["C"])
print(net)


Nodes: dict_keys(['A', 'B', 'C']), Edges: {'A': ['B'], 'B': ['C']}, Tables: {'A': {'Prior': (0.5, 0.5)}, 'B': {'Prior': (0.5, 0.5)}, 'C': {'Prior': (0.5, 0.5)}}


In [5]:
net.check_valid()

Node B probability table doesn't match parents' number of values
Node C probability table doesn't match parents' number of values


False

#### Checking Topological Sort

In [6]:
print(net.topo_sort())

['A', 'B', 'C']


In [7]:
net.add_prob_table("A", {"Prior" : (0.9, 0.1)})
net.add_prob_table("B", {"True" : (0.8, 0.2), "False" : (0.2, 0.8)})

#### Trying to add a probability table for a node not in the network

In [8]:
net.add_prob_table("D", {"Prior" : (0.9, 0.1)})

Node not in network


### Now we create a valid network

In [9]:
net.add_node("D")
net.add_edge("D", ["C"])
net.add_prob_table("C", {("True", "True") : (0.95, 0.05),
                          ("True", "False") : (0.7, 0.3),
                            ("False", "True") : (0.7, 0.3),
                            ("False", "False") : (0.1, 0.9)})
net.add_prob_table("D", {"Prior" : (0.9, 0.1)})
                          
print(net)
net.check_valid()

Nodes: dict_keys(['A', 'B', 'C', 'D']), Edges: {'A': ['B'], 'B': ['C'], 'D': ['C']}, Tables: {'A': {'Prior': (0.9, 0.1)}, 'B': {'True': (0.8, 0.2), 'False': (0.2, 0.8)}, 'C': {('True', 'True'): (0.95, 0.05), ('True', 'False'): (0.7, 0.3), ('False', 'True'): (0.7, 0.3), ('False', 'False'): (0.1, 0.9)}, 'D': {'Prior': (0.9, 0.1)}}


True

#### Let's try some single node sampling

In [10]:
probs = net.get_probabilities("A") # remember, we gave A probabilities of 0.9 for True, 0.1 for False
print(probs)
for i in range(20):
    print(net.sample(probs))

(0.9, 0.1)
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True


### Now we try Ancestral Sampling

In [11]:
samples = net.ancestral_sampling(10)
for i, sample in enumerate(samples):
    print(f"{i}: {sample}")

Topological sort: ['D', 'A', 'B', 'C']
0: {'D': 'True', 'A': 'True', 'B': 'True', 'C': 'True'}
1: {'D': 'True', 'A': 'True', 'B': 'True', 'C': 'True'}
2: {'D': 'True', 'A': 'True', 'B': 'True', 'C': 'True'}
3: {'D': 'True', 'A': 'True', 'B': 'True', 'C': 'True'}
4: {'D': 'True', 'A': 'True', 'B': 'True', 'C': 'True'}
5: {'D': 'True', 'A': 'True', 'B': 'True', 'C': 'True'}
6: {'D': 'True', 'A': 'True', 'B': 'True', 'C': 'True'}
7: {'D': 'True', 'A': 'True', 'B': 'False', 'C': 'True'}
8: {'D': 'True', 'A': 'True', 'B': 'True', 'C': 'True'}
9: {'D': 'False', 'A': 'True', 'B': 'False', 'C': 'False'}


# Modeling (some aspects of) my morning routine 

The network will represent the following structure:

In [12]:
daily_net = BayesianNetwork(nodes = {}, edges= {}, ordering= {}, values=["True", "False"])
daily_net.add_node("Noisy neighbors")
daily_net.add_node("Exams incoming")
daily_net.add_node("Bad day start")
daily_net.add_node("Call center call")
daily_net.add_node("Slept bad")
daily_net.add_node("Gig incoming")
daily_net.add_node("Sunny weather")
daily_net.add_node("Study")
daily_net.add_node("Play guitar")
daily_net.add_node("Clean")
daily_net.add_node("Gym")

In [13]:
daily_net.add_edge("Exams incoming", ["Gig incoming", "Study"])
daily_net.add_edge("Slept bad", ["Bad day start"])
daily_net.add_edge("Noisy neighbors", ["Bad day start"])
daily_net.add_edge("Call center call", ["Bad day start"])
daily_net.add_edge("Sunny weather", ["Bad day start"])
daily_net.add_edge("Bad day start", ["Play guitar", "Clean", "Gym"])
daily_net.add_edge("Gig incoming", ["Play guitar"])

In [14]:
# Determining prior probabilities
daily_net.add_prob_table("Exams incoming", {"Prior" : (0.1, 0.9)})
daily_net.add_prob_table("Noisy neighbors", {"Prior" : (0.3, 0.7)})
daily_net.add_prob_table("Slept bad", {"Prior" : (0.2, 0.8)})
daily_net.add_prob_table("Call center call", {"Prior" : (0.2, 0.8)})
daily_net.add_prob_table("Sunny weather", {"Prior" : (0.5, 0.5)})

In [15]:
daily_net.add_prob_table("Gig incoming", {"True" : (0.05, 0.95), "False" : (0.1, 0.9)}) # P(Gig incoming | Exams incoming)

daily_net.add_prob_table("Bad day start", {("True", "True", "True", "True") : (0.95, 0.05), # P(Bad day start | Slept bad, Noisy neighbors, Call center call, Sunny)
                                            ("False", "True", "True", "True") : (0.9, 0.1),
                                            ("True", "False", "True", "True") : (0.85, 0.15),
                                            ("True", "True", "False", "True") : (0.8, 0.2),
                                            ("True", "True", "True", "False") : (0.99, 0.01),
                                            ("False", "False", "True", "True") : (0.6, 0.4),
                                            ("True", "False", "False", "True") : (0.4, 0.6),
                                            ("True", "True", "False", "False") : (0.6, 0.4),
                                            ("False", "True", "False", "True") : (0.4, 0.6),
                                            ("True", "False", "True", "False") : (0.7, 0.3),
                                            ("False", "True", "True", "False") : (0.9, 0.1),
                                            ("False", "False", "False", "True") : (0.05, 0.95),
                                            ("True", "False", "False", "False") : (0.2, 0.8),
                                            ("False", "True", "False", "False") : (0.3, 0.7),
                                            ("False", "False", "True", "False") : (0.3, 0.7),
                                            ("False", "False", "False", "False") : (0.1, 0.9),    
                                            }) 

daily_net.add_prob_table("Study", {"True" : (0.9, 0.1), "False" : (0.2, 0.8)}) # P(Study | Exams incoming)

daily_net.add_prob_table("Play guitar", {("True", "True") : (0.7, 0.3), # P(Play guitar | Gig incoming, Bad day start)
                                          ("True", "False") : (0.9, 0.1),
                                          ("False", "True") : (0.2, 0.8),
                                          ("False", "False") : (0.6, 0.4)}) 
daily_net.add_prob_table("Clean", {"True" : (0.3, 0.7), "False" : (0.5, 0.5)}) # P(Clean | Bad day start)

daily_net.add_prob_table("Gym", {("True") : (0.3, 0.7), # P(Gym | Bad day start
                                 ("False") : (0.6, 0.4)})

In [16]:
# define ordering of parents for conditioned nodes wrt probability tables. Very dirty, should be done in a better way

daily_net.add_ordering({"Bad day start" : ("Slept bad", "Noisy neighbors", "Call center call", "Sunny weather"),
                        "Play guitar" : ("Gig incoming", "Bad day start")})

In [17]:
samples = daily_net.ancestral_sampling(100)
for i, sample in enumerate(samples):
    print(f"{i}: {sample}")

Topological sort: ['Sunny weather', 'Slept bad', 'Call center call', 'Exams incoming', 'Study', 'Gig incoming', 'Noisy neighbors', 'Bad day start', 'Gym', 'Clean', 'Play guitar']
0: {'Sunny weather': 'True', 'Slept bad': 'False', 'Call center call': 'True', 'Exams incoming': 'False', 'Study': 'False', 'Gig incoming': 'False', 'Noisy neighbors': 'False', 'Bad day start': 'True', 'Gym': 'False', 'Clean': 'False', 'Play guitar': 'False'}
1: {'Sunny weather': 'False', 'Slept bad': 'True', 'Call center call': 'False', 'Exams incoming': 'True', 'Study': 'True', 'Gig incoming': 'False', 'Noisy neighbors': 'True', 'Bad day start': 'False', 'Gym': 'False', 'Clean': 'True', 'Play guitar': 'True'}
2: {'Sunny weather': 'True', 'Slept bad': 'False', 'Call center call': 'False', 'Exams incoming': 'False', 'Study': 'False', 'Gig incoming': 'False', 'Noisy neighbors': 'False', 'Bad day start': 'False', 'Gym': 'False', 'Clean': 'True', 'Play guitar': 'True'}
3: {'Sunny weather': 'True', 'Slept bad': 'F